# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [16]:
%cd markdown_generator/
!pwd
!cat publications.tsv

[Errno 2] No such file or directory: 'markdown_generator/'
/home/mlindner/git/lindnemi.github.io/markdown_generator
/home/mlindner/git/lindnemi.github.io/markdown_generator
pub_date	title	category 	venue	excerpt	citation	url_slug	paper_url	slides_url
2025-12-01	Auswirkungen einer Anpassung der Ausbauziele für erneuerbare Energien für das Jahr 2030	Reports	Ariadne-Report	Diese Studie untersucht die Auswirkungen eines möglichen geringeren Ausbaus der Erneuerbaren Energien (EE) für das Jahr 2030 unter Unsicherheit bezüglich der Stromnachfrage.	Lindner, M., Geis, J., Luderer, G., Bartels, F., Brown, T. (2025): Auswirkungen einer Anpassung der Ausbauziele für erneuerbare Energien für das Jahr 2030 (Ariadne-Report), Potsdam: Potsdam Institute for Climate Impact Research, 20 p. https://doi.org/10.48485/pik.2025.030	auswirkungen-ee-ausbauziele-2030	https://publications.pik-potsdam.de/pubman/item/item_33514_2/component/file_33515/Ariadne-Report_EE-Ausbau_Dezember2025.pdf	
2025-10-22	Network Mea

## Import pandas

We are using the very handy pandas library for dataframes.

In [17]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [18]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,category,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2025-12-01,Auswirkungen einer Anpassung der Ausbauziele f...,Reports,Ariadne-Report,Diese Studie untersucht die Auswirkungen eines...,"Lindner, M., Geis, J., Luderer, G., Bartels, F...",auswirkungen-ee-ausbauziele-2030,https://publications.pik-potsdam.de/pubman/ite...,NaN
1,2025-10-22,Network Measure-Enriched GNNs: A New Framework...,Articles,IEEE Transactions on Knowledge and Data Engine...,Network measures integrated into GNNs improve ...,"Zhu, J., Nauck, C., Lindner, M., He, L., Yu, P...",network-measure-enriched-gnns-tkde-2025,https://doi.org/10.1109/TKDE.2025.3624222,NaN
2,2025-02-25,Climate change and class structure: greenhouse...,Articles,Innovation: The European Journal of Social Sci...,An ecological analysis of GHG emissions throug...,"Lindner, M., Dorschel, R., Schuster, A. (2025 ...",climate-change-class-structure-innovation-2025,https://publications.pik-potsdam.de/pubman/ite...,NaN
3,2025-01-01,Pypsa-De: Open-Source German Energy System Mod...,Conferences,European Energy Market (EEM),NaN,"Lindner, M., Geis, J., Seibold, T., Brown, T. ...",pypsa-de-eem-2025,https://arxiv.org/pdf/2510.09414,NaN
4,2025-01-01,Die Energiewende kosteneffizient gestalten: Sz...,Reports,Potsdam Institute for Climate Impact Research ...,NaN,"Luderer, G., Bartels, F., Brown, T., Aulich, C...",die-energiewende-kosteneffizient-klimaneutrali...,https://ariadneprojekt.de/publikation/report-s...,NaN
5,2025-01-01,A cost-efficient energy transition: Scenarios ...,Reports,Potsdam Institute for Climate Impact Research ...,NaN,"Luderer, G., Bartels, F., Brown, T., Aulich, C...",cost-efficient-energy-transition-2045,https://ariadneprojekt.de/en/publication/repor...,NaN
6,2024-08-21,Dirac-Bianconi Graph Neural Networks - Enablin...,Articles,Proceedings of Machine Learning Research (PMLR),DBGNNs enable coherent long-range message pass...,"Nauck, C., Gorantla, R., Lindner, M., Schürhol...",dirac-bianconi-gnn-2024,https://publications.pik-potsdam.de/pubman/ite...,NaN
7,2024-06-13,Predicting Fault-Ride-Through Probability of I...,Articles,arXiv,NaN,"Nauck, C., Büttner, A., Liemann, S., Hellmann,...",fault-ride-through-ml-arxiv-2406-08917,https://arxiv.org/pdf/2406.08917,NaN
8,2024-02-27,Predicting Instability in Complex Oscillator N...,Articles,arXiv,NaN,"Nauck, C., Lindner, M., Molkenthin, N., Kurths...",instability-oscillator-networks-arxiv-2402-17500,https://arxiv.org/pdf/2402.17500,NaN
9,2023-10-02,Toward dynamic stability assessment of power g...,Articles,Chaos,We provide large datasets for dynamic stabilit...,"Nauck, C., Lindner, M., Schürholt, K., Hellman...",toward-dynamic-stability-assessment-chaos-2023,https://publications.pik-potsdam.de/pubman/ite...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [19]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [20]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += f"""collection: publications"""

    md += "\ncategory: " + item.category
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

AttributeError: 'Series' object has no attribute 'category'

These files are in the publications directory, one directory below where we're working from.

In [15]:
!ls ../_publications/

2009-10-01-paper-title-number-1.md
2010-10-01-paper-title-number-2.md
2015-10-01-paper-title-number-3.md
2017-01-01-lagrangian-flow-network-vortex-flow-2017.md
2019-01-01-characterizing-flows-complex-network-methods-2019.md
2019-01-01-stochastic-basins-committor-functions-2019.md
2021-06-22-networkdynamics-jl-chaos-2021.md
2022-04-01-predicting-basin-stability-njp-2022.md
2022-05-09-open-source-software-stack-osmses-2022.md
2022-12-21-towards-dynamic-stability-arxiv-2212-11130.md
2023-01-01-renewable-energy-transition-thesis-2023.md
2023-05-01-whose-house-is-on-fire-ecolecon-2023.md
2023-10-02-toward-dynamic-stability-assessment-chaos-2023.md
2024-02-17-paper-title-number-4.md
2024-02-27-instability-oscillator-networks-arxiv-2402-17500.md
2024-06-13-fault-ride-through-ml-arxiv-2406-08917.md
2024-08-21-dirac-bianconi-gnn-2024.md
2025-01-01-cost-efficient-energy-transition-2045.md
2025-01-01-die-energiewende-kosteneffizient-klimaneutralitaet-2045.md
2025-01-01-pypsa-de-eem-2025.md
2025-0

In [16]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
slidesurl: 'http://academicpages.github.io/files/slides1.pdf'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download slides here](http://academicpages.github.io/files/slides1.pdf)

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).